In [1]:
import pandas as pd
import re

In [2]:
bank_statement_df = pd.read_excel(r"D:\NPS_phase1\BANK STMT PHASE 4\Bank_sanima.xlsx", skiprows=1)


In [3]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,90801352150025,100.0,C,"100003986533,MOBILE/009011040000173",FUN MOB/9808006822/277055/28-08-2023,NaN,2023-08-28T19:48:44,NaN
1,2,90801352150025,-1980.0,D,NPS-IF-3133560/027011040000037,FUN APS/308213/125273/28-08-2023,NaN,2023-08-28T19:44:53,NaN
2,3,90801352150025,-86120.0,D,NPS-IF-3133552/019011090001381,FUN APS/308213/125272/28-08-2023,NaN,2023-08-28T19:43:54,NaN
3,4,90801352150025,-4000.0,D,NPS-IF-3133525/011011090003118,FUN APS/308213/125271/28-08-2023,NaN,2023-08-28T19:36:56,NaN
4,5,90801352150025,-1000.0,D,NPS-IF-3133517/106011060001623,FUN APS/308213/125270/28-08-2023,NaN,2023-08-28T19:35:50,NaN


In [6]:
for index, row in bank_statement_df.iterrows():
    # print(bank_statement_df.iloc[index]['Transaction ID'])
    for roww in (row['Desc2'] , row['Desc1']):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            break
        
        
        elif "10000" in roww:
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            
        elif re.findall(r'\d{7}-.*', roww):
            id = re.findall(r'[0-9]{7}', roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]

In [7]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id
0,1,90801352150025,100.0,C,"100003986533,MOBILE/009011040000173",FUN MOB/9808006822/277055/28-08-2023,NaN,2023-08-28T19:48:44,NaN,3986533
1,2,90801352150025,-1980.0,D,NPS-IF-3133560/027011040000037,FUN APS/308213/125273/28-08-2023,NaN,2023-08-28T19:44:53,NaN,3133560
2,3,90801352150025,-86120.0,D,NPS-IF-3133552/019011090001381,FUN APS/308213/125272/28-08-2023,NaN,2023-08-28T19:43:54,NaN,3133552
3,4,90801352150025,-4000.0,D,NPS-IF-3133525/011011090003118,FUN APS/308213/125271/28-08-2023,NaN,2023-08-28T19:36:56,NaN,3133525
4,5,90801352150025,-1000.0,D,NPS-IF-3133517/106011060001623,FUN APS/308213/125270/28-08-2023,NaN,2023-08-28T19:35:50,NaN,3133517


In [8]:
bank_statement_df['Transaction Id'].count()

379

In [9]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'D' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'C':
            mode = 'CR'
            amount = row['Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [10]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [11]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,90801352150025,100.0,C,"100003986533,MOBILE/009011040000173",FUN MOB/9808006822/277055/28-08-2023,NaN,2023-08-28T19:48:44,NaN,3986533,CR
1,2,90801352150025,1980.0,D,NPS-IF-3133560/027011040000037,FUN APS/308213/125273/28-08-2023,NaN,2023-08-28T19:44:53,NaN,3133560,DR
2,3,90801352150025,86120.0,D,NPS-IF-3133552/019011090001381,FUN APS/308213/125272/28-08-2023,NaN,2023-08-28T19:43:54,NaN,3133552,DR
3,4,90801352150025,4000.0,D,NPS-IF-3133525/011011090003118,FUN APS/308213/125271/28-08-2023,NaN,2023-08-28T19:36:56,NaN,3133525,DR
4,5,90801352150025,1000.0,D,NPS-IF-3133517/106011060001623,FUN APS/308213/125270/28-08-2023,NaN,2023-08-28T19:35:50,NaN,3133517,DR


In [12]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [13]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,90801352150025,100.0,C,"100003986533,MOBILE/009011040000173",FUN MOB/9808006822/277055/28-08-2023,NaN,2023-08-28,NaN,3986533,CR
1,2,90801352150025,1980.0,D,NPS-IF-3133560/027011040000037,FUN APS/308213/125273/28-08-2023,NaN,2023-08-28,NaN,3133560,DR
2,3,90801352150025,86120.0,D,NPS-IF-3133552/019011090001381,FUN APS/308213/125272/28-08-2023,NaN,2023-08-28,NaN,3133552,DR
3,4,90801352150025,4000.0,D,NPS-IF-3133525/011011090003118,FUN APS/308213/125271/28-08-2023,NaN,2023-08-28,NaN,3133525,DR
4,5,90801352150025,1000.0,D,NPS-IF-3133517/106011060001623,FUN APS/308213/125270/28-08-2023,NaN,2023-08-28,NaN,3133517,DR
